In [31]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import json
import csv
import pandas
import re
from pathlib import Path
from datetime import datetime
from pandas.io.json import json_normalize 
import matplotlib.pyplot as plt 

In [17]:
with open('output/losangeles.eventful.com!events!categories!music.json') as f: 
    d = json.load(f) 
    df = json_normalize(d["elements"]) 

    df.fontSize = df.fontSize.apply(lambda x : re.sub("[^0-9]", "", x))
    df.lineHeight = df.lineHeight.apply(lambda x : re.sub("[^0-9]", "", x))
    df.itemType = df.itemType.apply(lambda x : "http://schema.org/Event" == x)

    y = df.itemType
    X = df.drop(labels=["color", "elementId", "tagName", "classList", "textContent", "font", "fontStyle", "backgroundColor", "background", "backgroundImage", "itemType", "itemProp"], axis=1)

    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

    print("Whole set:", X.shape, y.shape)
    print("Training set:", xTrain.shape, yTrain.shape)
    print("Testing set:", xTest.shape, yTest.shape)



Whole set: (1411, 8) (1411,)
Training set: (1128, 8) (1128,)
Testing set: (283, 8) (283,)
UsageError: Line magic function `%%bash` not found.


In [20]:
%%bash
aws s3 cp s3://html-feature-extractor-worker-output/C2Z5BEoK/ output/s3 --recursive --exclude "*" --include "*.json" --profile bmccarthy

download: s3://html-feature-extractor-worker-output/C2Z5BEoK/losangeles.eventful.com!events!categories!music_Icpa7Ui7.json to output/test/losangeles.eventful.com!events!categories!music_Icpa7Ui7.json


In [24]:
pathlist = Path('output/test').glob('**/*.json')
for path in pathlist:
    path_in_str = str(path)
    with open(path_in_str) as f: 
        d = json.load(f) 
        df = json_normalize(d["elements"]) 



In [49]:
now = datetime.now()
s3_prefix = now.strftime("%Y_%m_%d_%H_%M_%S")
print("s3_prefix =", s3_prefix)



s3_prefix = 2020_01_28_12_00_17


In [48]:
# Now run populate-queue to take the unique list of urls and send them to be parsed by the worker

In [50]:
%%bash
export AWS_PROFILE="bmccarthy" && node src/populate-queue.js -i output/unique_url.txt -p s3_prefix -c 2


{ verbose: false,
  input: 'output/unique_url.txt',
  count: '2',
  prefix: 's3_prefix' }
DONE!
Finished sending 2 messages to https://sqs.us-east-1.amazonaws.com/109628666462/html-feature-extractor-worker-queue. Ending now..
